<a href="https://colab.research.google.com/github/sid-mishra910/Fake-News-Detection/blob/master/Fake_News_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
cd drive/My Drive/News Detection/

/content/drive/My Drive/News Detection


In [3]:
import pandas as pd
import re
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [4]:
real = pd.read_csv('True.csv')
fake = pd.read_csv('Fake.csv')

In [11]:
real.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [12]:
fake.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [14]:
real.loc[real.text.str.contains('Reuters')].count()/real.count()

title      0.998179
text       0.998179
subject    0.998179
date       0.998179
dtype: float64

In [16]:
fake.loc[fake.text.str.contains('Reuters')].count()/fake.count()

title      0.013245
text       0.013245
subject    0.013245
date       0.013245
dtype: float64

In [17]:
fake['fake'] = 1
real['fake'] = 0
combined_df = pd.concat([fake,real])

In [18]:
combined_df.head()

,title,text,subject,date,fake
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",1
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",1
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",1
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",1
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",1


In [19]:
X_train, X_test, Y_train, Y_test = train_test_split(combined_df['text'],combined_df['fake'], random_state = 42)

In [24]:
X_train.head()

11349    SEOUL (Reuters) - South Korea predicted on Tue...
6018     Beebe, Arkansas mayor, Mike Robertson, loves J...
19068    NEW YORK (Reuters) - Eighty percent of the pow...
8670     When the Black Lives Matter movement started g...
3762     WASHINGTON (Reuters) - The Trump administratio...
Name: text, dtype: object

In [25]:
X_test.head()

22216    21st Century Wire says Ben Stein, reputable pr...
4436     WASHINGTON (Reuters) - U.S. President Donald T...
1526     (Reuters) - Puerto Rico Governor Ricardo Rosse...
1377     On Monday, Donald Trump once again embarrassed...
8995     GLASGOW, Scotland (Reuters) - Most U.S. presid...
Name: text, dtype: object

In [26]:
Y_train.head()

11349    0
6018     1
19068    0
8670     1
3762     0
Name: fake, dtype: int64

In [27]:
max_words = 2000
max_len = 400

token = Tokenizer(num_words= max_words, lower = True, split =' ')

In [29]:
token.fit_on_texts(X_train.values)

In [38]:
print(token.word_index)

{'the': 1, 'to': 2, 'of': 3, 'a': 4, 'and': 5, 'in': 6, 'that': 7, 's': 8, 'on': 9, 'for': 10, 'is': 11, 'said': 12, 'he': 13, 'it': 14, 'trump': 15, 'with': 16, 'was': 17, 'as': 18, 'by': 19, 'his': 20, 'has': 21, 'be': 22, 'have': 23, 'not': 24, 'from': 25, 'this': 26, 'at': 27, 'are': 28, 'who': 29, 'an': 30, 'they': 31, 'but': 32, 'i': 33, 'we': 34, 'would': 35, 'president': 36, 'u': 37, 'about': 38, 'will': 39, 't': 40, 'their': 41, 'you': 42, 'had': 43, 'been': 44, 'people': 45, 'were': 46, '”': 47, 'or': 48, 'more': 49, 'which': 50, 'she': 51, 'one': 52, 'after': 53, 'her': 54, 'all': 55, 'if': 56, 'out': 57, 'state': 58, 'what': 59, 'when': 60, 'new': 61, 'also': 62, 'up': 63, 'its': 64, 'there': 65, 'no': 66, 'reuters': 67, 'over': 68, 'so': 69, 'donald': 70, 'government': 71, 'house': 72, 'our': 73, 'clinton': 74, 'can': 75, 'states': 76, 'obama': 77, 'republican': 78, 'him': 79, 'than': 80, 'other': 81, 'just': 82, 'some': 83, 'year': 84, 'could': 85, 'united': 86, 'told': 8

In [34]:
seq = token.texts_to_sequences(X_train.values)

In [36]:
print(seq[:100])

[[67, 309, 254, 9, 194, 7, 171, 254, 35, 410, 2, 494, 1260, 16, 1, 86, 76, 233, 84, 6, 30, 10, 1243, 110, 18, 427, 63, 4, 178, 491, 2, 283, 1168, 25, 1, 171, 1, 37, 381, 124, 567, 61, 468, 9, 171, 254, 9, 214, 10, 64, 360, 1985, 754, 1246, 4, 361, 1, 171, 30, 394, 5, 354, 3, 212, 171, 254, 39, 1197, 16, 86, 76, 111, 2, 64, 622, 2, 22, 18, 4, 1269, 283, 112, 309, 254, 8, 732, 12, 6, 4, 264, 272, 102, 1523, 10, 64, 1, 732, 3, 12, 14, 35, 342, 2, 145, 4, 61, 235, 171, 254, 219, 1499, 2, 5, 1040, 2, 171, 254, 8, 283, 5, 754, 692, 1561, 23, 68, 171, 254, 8, 283, 5, 754, 1036, 50, 14, 6, 3, 119, 3, 37, 381, 124, 567, 16, 1274, 609, 25, 199, 1539, 5, 1, 91, 72, 37, 8, 1921, 23, 129, 333, 31, 28, 976, 4, 1226, 1714, 32, 36, 70, 15, 21, 480, 18, 5, 12, 1539, 325, 2, 1004, 63, 64, 283, 598, 118, 102, 480, 75, 1497, 6, 4, 181, 1603, 19, 1, 235, 101, 335, 171, 254, 12, 1, 86, 76, 17, 19, 64, 283, 463, 5, 17, 601, 49, 5, 49, 6, 1, 2, 1, 420, 468, 5, 883, 9, 73, 112, 220, 1, 171, 8, 373, 1492, 5, 15

In [39]:
train_seq_padd = pad_sequences(seq, maxlen = max_len)

In [40]:
print(train_seq_padd)

[[ 171  254  219 ...  171   18   10]
 [  20  575 1153 ...  156  235  922]
 [   0    0    0 ... 1512   19  525]
 ...
 [   0    0    0 ...   11    4 1273]
 [   7   15 1006 ...  156  574  503]
 [   0    0    0 ...  156 1065  101]]


In [41]:
embedding_dim = 50
lstm_out = 64
batch_size = 32

model = Sequential()
model.add(Embedding(max_words, embedding_dim, input_length= max_len))
model.add(LSTM(lstm_out))
model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dense(1, name='out_layer'))
model.add(Activation('sigmoid'))

model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 400, 50)           100000    
_________________________________________________________________
lstm (LSTM)                  (None, 64)                29440     
_________________________________________________________________
dense (Dense)                (None, 256)               16640     
_________________________________________________________________
activation (Activation)      (None, 256)               0         
_________________________________________________________________
out_layer (Dense)            (None, 1)                 257       
_________________________________________________________________
activation_1 (Activation)    (None, 1)                 0         
Total params: 146,337
Trainable params: 146,337
Non-trainable params: 0
__________________________________________________

In [48]:
hist = model.fit(train_seq_padd, Y_train, batch_size= batch_size, epochs = 5, validation_split=0.2)

Epoch 1/5
842/842 [==============================] - 216s 257ms/step - loss: 0.0324 - accuracy: 0.9904 - val_loss: 0.0384 - val_accuracy: 0.9877
Epoch 2/5
842/842 [==============================] - 216s 256ms/step - loss: 0.0172 - accuracy: 0.9952 - val_loss: 0.0507 - val_accuracy: 0.9887
Epoch 3/5
842/842 [==============================] - 215s 256ms/step - loss: 0.0522 - accuracy: 0.9804 - val_loss: 0.0342 - val_accuracy: 0.9893
Epoch 4/5
842/842 [==============================] - 216s 257ms/step - loss: 0.0126 - accuracy: 0.9957 - val_loss: 0.0203 - val_accuracy: 0.9933
Epoch 5/5
842/842 [==============================] - 216s 256ms/step - loss: 0.0049 - accuracy: 0.9987 - val_loss: 0.0112 - val_accuracy: 0.9975


In [44]:
test_seq = token.texts_to_sequences(X_test)

In [45]:
test_seq_pad = pad_sequences(test_seq, maxlen=max_len)

In [46]:
model.evaluate(test_seq_pad, Y_test)

351/351 [==============================] - 19s 55ms/step - loss: 0.0405 - accuracy: 0.9882


[0.04048451408743858, 0.9882405400276184]

In [47]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.legend(['Training', 'Validation'])
plt.ylabel('Accuracy (%)')
plt.xlabel('Epochs')
plt.xticks([0,1,2,3,4])

NameError: ignored